In [38]:
import torch; torch.manual_seed(42)
from torch.nn.functional import gelu

In [39]:
X = torch.randn(4, 2, device="cpu", dtype=torch.float32)
W = torch.randn(2, 2, device="cpu", dtype=torch.float32)
gelu(X @ W)

tensor([[ 0.1321,  0.1121],
        [ 0.1369,  0.1491],
        [-0.1658, -0.1470],
        [ 0.5094,  0.0392]])

In [40]:
# Column linear

W_0, W_1 = W.chunk(2, dim=1)

y_col_1 = torch.cat([gelu(X @ W_0), gelu(X @ W_1)], dim=1)

y_col_2 = gelu(torch.cat([X @ W_0, X @ W_1], dim=1))

try:
    torch.testing.assert_close(y_col_1, y_col_2, rtol=1e-5, atol=1e-5)
    col_match = True
    print("Column linear match")
except AssertionError:
    col_match = False
    print("Column linear mismatch")

y_col_2

Column linear match


tensor([[ 0.1321,  0.1121],
        [ 0.1369,  0.1491],
        [-0.1658, -0.1470],
        [ 0.5094,  0.0392]])

In [41]:
# Row linear

X_0, X_1 = X.chunk(2, dim=1)
W_0, W_1 = W.chunk(2, dim=0)

y_row_1 = gelu(X_0 @ W_0) + gelu(X_1 @ W_1)
y_row_2 = gelu(X_0 @ W_0 + X_1 @ W_1)

try:
    torch.testing.assert_close(y_row_1, y_row_2, rtol=1e-5, atol=1e-5)
    col_match = True
    print("Row linear match")
except AssertionError:
    col_match = False
    print("Row linear mismatch")

print(y_row_1, "\n\n", y_row_2)

Row linear mismatch
tensor([[ 0.1237,  0.1047],
        [ 0.1264,  0.1399],
        [-0.2025, -0.1810],
        [ 0.7374,  0.2702]]) 

 tensor([[ 0.1321,  0.1121],
        [ 0.1369,  0.1491],
        [-0.1658, -0.1470],
        [ 0.5094,  0.0392]])
